<a href="https://colab.research.google.com/github/Loinen/datagen/blob/main/SDVmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sdv

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import warnings
import random as rd

from sklearn.metrics import f1_score, accuracy_score, roc_auc_score,  plot_roc_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, StratifiedShuffleSplit
from sklearn import model_selection
from sklearn.metrics import classification_report, plot_confusion_matrix

In [8]:
from sdv.timeseries import PAR

ValueError: ignored

In [ ]:
data = pd.read_csv("base25data/Shestakovo_validation2.csv", index_col=0)
data.LITHO = data.LITHO.astype('int8') # data.LITHO.astype('category')

In [ ]:
# Разобьем данные на тренировочную и тестовую части в пропорции 0.7 и 0.3 соответственно
train_part_size = 0.7
# Случайно выберем скважины для тренировочного и тестового наборов
rd.seed(17)
train_wells = rd.sample(data.Well.unique().tolist(), round(len(data.Well.unique())*train_part_size))
train_set = data.loc[data.Well.isin(train_wells)]
test_set = data.loc[data.Well.isin(i for i in data.Well.unique() if i not in train_wells)]

Feature_train = train_set.drop(['DEPT', 'Well', 'LITHO'], axis=1)
Target_train = train_set['LITHO']
Feature_test = test_set.drop(['DEPT', 'Well', 'LITHO'], axis=1)
Target_test = test_set['LITHO']

In [ ]:
# baseline
clf = RandomForestClassifier(class_weight="balanced")
clf.fit(Feature_train, Target_train)
predicted_litho = clf.predict(Feature_test)

# Посчитаем точность полученную нашей baseline-моделью
# Считаться будут две метрики - F1 и Accuracy
base_acc = accuracy_score(Target_test, predicted_litho)
base_f1 = f1_score(Target_test, predicted_litho)
print('F1 метрика равна %.3f, и доля правильных ответов составляет %.3f для базового кейса.'%(base_f1, base_acc))

In [ ]:
# PAR model

entity_columns = ['Well']
sequence_index = 'DEPT'

model = PAR(
    entity_columns=entity_columns,
    sequence_index=sequence_index,
)

model.fit(train_set)  # учим на том же, на чем учили KN
model.save('par_base_kn_l200.pkl')

# 0.034 - 200, 0.03 - base, 400 - 0.51б 300 -
model = PAR.load('par_base_kn_l200.pkl')
new_data = model.sample(470, sequence_length=300)  # 140k
new_data.to_csv('par_results300_1.csv', index=False)
new_data = pd.read_csv("par_results200.csv", index_col=False)

model = PAR.load('par_base_kn_l200.pkl')
new_data = model.sample(1100)  # 1 = 160 значений
new_data.to_csv('par_results_200_no_ns.csv', index=False)

In [ ]:
# new_data = pd.read_csv("par_results_200_no_ns.csv", index_col=False)

In [ ]:
# отрисуем полученные данные
new_data[:10000].plot(subplots=True, figsize=(15, 10))
plt.ylabel("Value syn")
plt.xlabel("N")
plt.legend(loc='best')
plt.xticks(rotation='vertical')

In [ ]:
# отрисуем обучающие данные
train_set.plot(subplots=True, figsize=(15, 10))
plt.ylabel("Value real")
plt.xlabel("N")
plt.legend(loc='best')
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
print(new_data.shape)
print(new_data.head())

In [ ]:

new_data = new_data.drop(np.where(new_data['LITHO'] > 1)[0])
new_data = new_data.replace(-1, np.nan, regex=True)
new_data = new_data.dropna(subset=['LITHO'])

new_data.LITHO = new_data.LITHO.values.astype('bool')  # костыль
new_data.LITHO = new_data.LITHO.values.astype('int')
pd.set_option('display.max_columns', 15)
print(train_set.describe())
print(new_data.describe())

par_feature_train = new_data.drop(['DEPT', 'Well', 'LITHO'], axis=1)
par_target_train = new_data['LITHO']
print("new shape", new_data.shape)

print(par_target_train.value_counts('LITHO'))
print(train_set.value_counts('LITHO'))
print("test", test_set.value_counts('LITHO'))

In [ ]:
sample_weight = par_target_train.shape[0]/(2*np.bincount(par_target_train))

rf = {"clf__criterion":["gini", "entropy"], "clf__max_depth": range(2,8,2),
           #"clf__class_weight":[{0:1, 1:75}, {0:1, 1:65}, {0:1, 1:70}],
           "clf__min_samples_leaf": range(1,7,1), "clf__n_estimators":range(50,90,20)
      }

pipe = Pipeline(steps=[ ('scaler', MinMaxScaler()),
                            ('clf', RandomForestClassifier(random_state=42))
                      ])
grid_cv = model_selection.GridSearchCV(pipe, rf, scoring='roc_auc')
grid_cv.fit(par_feature_train, par_target_train, clf__sample_weight=sample_weight[par_target_train])
print("model best score: ", grid_cv.best_score_)
print(grid_cv.best_params_)
clf_par = grid_cv.best_estimator_


In [ ]:
par_prediction = clf_par.predict(Feature_test)
user_acc = accuracy_score(Target_test, par_prediction)
user_f1 = f1_score(Target_test, par_prediction)
print('F1 метрика равна %.3f, и доля правильных ответов составляет %.3f для базового кейса на PAR.'%
      (user_f1, user_acc))

In [ ]:
# отрисуем полученные данные (1000)
new_data[:5000].plot(subplots=True, figsize=(15, 10))
plt.ylabel("Value syn")
plt.xlabel("N")
plt.legend(loc='best')
plt.xticks(rotation='vertical')
plt.show()

In [ ]:

# отрисуем полученные данные (1000)
new_data[:300].plot(subplots=True, figsize=(15, 10))
plt.ylabel("Value syn")
plt.xlabel("N")
plt.legend(loc='best')
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
# отрисуем обучающие данные (1000)
train_set[:5000].plot(subplots=True, figsize=(15, 10))
plt.ylabel("Value real")
plt.xlabel("N")
plt.legend(loc='best')
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
print(classification_report(par_prediction, Target_test))
print(classification_report(predicted_litho, Target_test))

In [ ]:
print("roc_auc real data ", roc_auc_score(Target_test, predicted_litho),
      "roc_auc syn data ", roc_auc_score(Target_test, par_prediction))

In [ ]:
plot_confusion_matrix(clf_par, Feature_test, Target_test, values_format='d')

In [ ]:
plot_confusion_matrix(clf, Feature_test, Target_test, values_format='d')

In [ ]:
sns.boxplot(data=new_data[["SP", "GR", "DT", "DENS"]])

In [ ]:
sns.boxplot(data=train_set[["SP", "GR", "DT", "DENS"]])

In [ ]:
plot_roc_curve(clf_par, Feature_test, Target_test)

In [ ]:
plot_roc_curve(clf, Feature_test, Target_test)